In [2]:
from serlo import SerloBot, Environment

bot = SerloBot(Environment.STAGING)
bot.login()

import import_ipynb
from helper_functions import get_newest_unrevised_revision, save_revision, select_all_entities
from boxes import delete_empty_boxes

import json

Serlo-Username: ········
Serlo-Password: ········


AttributeError: 'NoneType' object has no attribute 'select'

In [ ]:
def get_potential_entites_to_be_updated():
    def has_empty_boxes(revision):
        try:
            currentObject = json.loads(revision["value"])
        except:
            return False
        
        return isinstance(currentObject, dict) and delete_empty_boxes(currentObject)[1] > 0
    
    return [revision["id"] for revision in select_all_entities().iterrows() if has_empty_boxes(revision)]

def apply_rules(state):
    #[state, number_of_deleted_equations] = delete_empty_equations(new_revision)
    [state, number_of_deleted_boxes] = delete_empty_boxes(state, 0)
    rules = ""
    #if number_of_deleted_equations:
    #    rules.concat("Deleted {number_of_deleted_equations} empty equations.")
    print("#found_empty_boxes:")
    print(number_of_deleted_boxes)
    if number_of_deleted_boxes == 1:
        rules += "Deleted 1 empty box."
    elif number_of_deleted_boxes:
        rules += "Deleted %s empty boxes." % number_of_deleted_boxes
        
    return {"new_revision": state, "rules": rules}

In [ ]:
entities_to_be_updated = get_potential_entites_to_be_updated()
print("to be updated")
print(entities_to_be_updated)

for entity in entities_to_be_updated:
    newest_unrevised_revision = get_newest_unrevised_revision(entity, bot)
    json_readable_content = newest_unrevised_revision["revision"]["content"].replace("'", "\"")
    content = json.loads(json_readable_content)

    if newest_unrevised_revision["revision"]["trashed"]:
        continue

    new_revision = apply_rules(content.copy())

    if new_revision["new_revision"] != content:
        save_revision(
            entity,
            new_revision["new_revision"],
            new_revision["rules"],
            newest_unrevised_revision,
            bot
        )